In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [2]:
import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part").map(parse_article)

In [3]:
def word_pair(words, first_word):
    rez = []
    
    for i in range(0, len(words) - 1):
        if words[i].lower() == first_word:
            rez.append((words[i].lower() + '_' + words[i+1].lower(), 1))
    
    return rez

In [4]:
wiki_pairs = wiki.flatMap(lambda words: word_pair(words, 'narodnaya')) \
    .filter(lambda x: x != []) \
    .reduceByKey(lambda a, b: a + b)

In [6]:
rez = wiki_pairs.collect()

In [7]:
for pair, cnt in rez:
    print '{}\t{}'.format(pair, cnt)

narodnaya_gazeta	1
narodnaya_volya	9


In [8]:
sc.stop()